In [66]:
import pandas as pd

In [67]:
sql = (
    '''
    WITH
      enbloc_list AS (
      SELECT
        project_name,
        EXTRACT(YEAR FROM contract_date) AS contract_year,
        1 AS enbloc
      FROM
        `drq-machine-learning.enbloc.enbloc`),
      property AS (
      SELECT
        UPPER(property_name) AS project_name,
        * EXCEPT(property_name)
      FROM
        `drq-machine-learning.enbloc.property_list` )
    SELECT
      property.*,
      enbloc_list.contract_year,
      enbloc_list.enbloc
    FROM
      property
    LEFT JOIN
      enbloc_list
    ON
      property.project_name = enbloc_list.project_name
    '''
)



In [68]:
data = pd.read_gbq(
    sql,
    project_id='413980110872',
    private_key='session2/drq-machine-learning-0a5d30a93870.json',
    dialect='standard',
    verbose=True
)

In [69]:
data.to_csv('enbloc_property.csv', index=False)

In [71]:
data = data[pd.notnull(data['TOP'])]

In [72]:
data['units'] = data['units'].fillna(data['units'].median())

In [73]:
data = data[pd.notnull(data['tenure'])]

In [74]:
data['enbloc'] = data['enbloc'].fillna(0)

In [75]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1776 entries, 4 to 1834
Data columns (total 10 columns):
project_name     1776 non-null object
address          1776 non-null object
TOP              1776 non-null object
developer        1776 non-null object
district         1776 non-null object
units            1776 non-null float64
property_type    1776 non-null object
tenure           1776 non-null object
contract_year    17 non-null object
enbloc           1776 non-null int64
dtypes: float64(1), int64(1), object(8)
memory usage: 152.6+ KB


In [76]:
data['contract_year'] = data['contract_year'].fillna(2018)

In [77]:
data

,project_name,address,TOP,developer,district,units,property_type,tenure,contract_year,enbloc
4,PARKSUITES,"28 Holland Grove Road, 278805",1900,N.A.,"D10 - Bukit Timah, Holland Rd, Tanglin",87.0,Apartment,110 years from 01/11/2017,2018,0
5,TEACHERS' HOUSING ESTATE,"1 Tu Fu Avenue, 787209",1968,N.A.,"D26 - Upper Thomson, Springleaf",87.0,Apartment,999 Years,2018,0
6,LAKESIDE APARTMENTS,"9E Yuan Ching Road, 618647",1970,N.A.,D22 - Jurong,87.0,Apartment,99 years from 01/03/1977,2018,0
7,PEOPLE'S PARK COMPLEX,"1 Park Road, 059108",1972,N.A.,"D1 - Temasek Blvd, Raffles Link",87.0,Apartment,99 years from 25/03/1968,2018,0
8,GOLDEN MILE COMPLEX,"5001 Beach Road, 199588",1974,N.A.,"D7 - Middle Road, Golden Mile",87.0,Apartment,99 years from 04/08/1969,2018,0
9,INTERNATIONAL PLAZA,"10 Anson Road, 079903",1976,International Associated Co Pte Ltd,"D2 - Anson, Tanjong Pagar",87.0,Apartment,99 years from 02/06/1970,2018,0
10,FOOK HAI BUILDING,"150 South Bridge Road, 058727",1976,N.A.,"D1 - Temasek Blvd, Raffles Link",87.0,Apartment,99 years from 18/01/1972,2018,0
11,PEOPLE'S PARK CENTRE,"101 Upper Cross Street, 058357",1976,N.A.,"D1 - Temasek Blvd, Raffles Link",87.0,Apartment,99 years from 02/06/1970,2018,0
12,TEXTILE CENTRE,"200 Jalan Sultan, 199018",1977,N.A.,"D7 - Middle Road, Golden Mile",87.0,Apartment,99 years from 02/06/1970,2018,0
13,PEACE CENTRE/MANSIONS,"1 Sophia Road, 228149",1977,KIAN AN REALTY LTD,"D9 - Orchard, Cairnhill, River Valley",42.0,Apartment,99 years from 02/06/1970,2018,0


In [78]:
data['tenure'].unique()

array(['110 years from 01/11/2017', '999 Years',
       '99 years from 01/03/1977', '99 years from 25/03/1968',
       '99 years from 04/08/1969', '99 years from 02/06/1970',
       '99 years from 18/01/1972', '99 years from 03/09/1968',
       '99 years from 01/01/1969', '99 years from 30/08/1976',
       '99 years from 01/11/1975', '999 years from 07/06/1884',
       '99 years from 18/09/1979', '99 years from 16/06/1980',
       '99 years from 15/12/1980', '999 years from 25/01/1827',
       '9999 years from 02/06/1995', '999 years from 29/05/1885',
       '99 years from 08/09/1992', '99 years from 01/01/1995',
       '999 years from 02/06/1882', '99 years from 17/11/1993',
       '99 years from 02/10/1992', 'LEASEHOLD/99 years',
       '929 years from 01/01/1953', '99 years from 13/02/1995',
       '99 years from 06/11/1995', '99 years from 06/09/1996',
       '99 years from 10/06/1996', '99 years from 01/12/1994',
       '99 years from 01/05/1994', '999 years from 12/10/1885',
    

In [79]:
data['tenure'] = data['tenure'].apply(lambda x: x.split(' ')[0])

In [80]:
data['district'] = data['district'].apply(lambda x: x.split('-')[0])

In [81]:
data

,project_name,address,TOP,developer,district,units,property_type,tenure,contract_year,enbloc
4,PARKSUITES,"28 Holland Grove Road, 278805",1900,N.A.,D10,87.0,Apartment,110,2018,0
5,TEACHERS' HOUSING ESTATE,"1 Tu Fu Avenue, 787209",1968,N.A.,D26,87.0,Apartment,999,2018,0
6,LAKESIDE APARTMENTS,"9E Yuan Ching Road, 618647",1970,N.A.,D22,87.0,Apartment,99,2018,0
7,PEOPLE'S PARK COMPLEX,"1 Park Road, 059108",1972,N.A.,D1,87.0,Apartment,99,2018,0
8,GOLDEN MILE COMPLEX,"5001 Beach Road, 199588",1974,N.A.,D7,87.0,Apartment,99,2018,0
9,INTERNATIONAL PLAZA,"10 Anson Road, 079903",1976,International Associated Co Pte Ltd,D2,87.0,Apartment,99,2018,0
10,FOOK HAI BUILDING,"150 South Bridge Road, 058727",1976,N.A.,D1,87.0,Apartment,99,2018,0
11,PEOPLE'S PARK CENTRE,"101 Upper Cross Street, 058357",1976,N.A.,D1,87.0,Apartment,99,2018,0
12,TEXTILE CENTRE,"200 Jalan Sultan, 199018",1977,N.A.,D7,87.0,Apartment,99,2018,0
13,PEACE CENTRE/MANSIONS,"1 Sophia Road, 228149",1977,KIAN AN REALTY LTD,D9,42.0,Apartment,99,2018,0


In [83]:
data[data['tenure'] == 'LEASEHOLD/99']

,project_name,address,TOP,developer,district,units,property_type,tenure,contract_year,enbloc
32,CAVENAGH LODGE,81 Cavenagh Road,1996,N.A.,D9,87.0,Apartment,LEASEHOLD/99,2018,0
142,HIGH STREET CENTRE,1 North Bridge Road,1969,N.A.,D6,87.0,Condominium,LEASEHOLD/99,2018,0
365,AVANT RESIDENCES,10 Aljunied Road,2012,N.A.,D14,50.0,Condominium,LEASEHOLD/99,2018,0


In [84]:
data['tenure'].apply(lambda x: x.replace('LEASEHOLD/99', '99')).unique()

array(['110', '999', '99', '9999', '929', '947', '946', '956', '60',
       'freehold', '101', '100', '103', '99-year', 'N.A', 'Mixed',
       'FREEHOLD', 'Freehold'], dtype=object)

In [85]:
data['tenure'].apply(lambda x: x.replace('Freehold', 'FREEHOLD')).unique()

array(['110', '999', '99', '9999', 'LEASEHOLD/99', '929', '947', '946',
       '956', '60', 'freehold', '101', '100', '103', '99-year', 'N.A',
       'Mixed', 'FREEHOLD'], dtype=object)

In [86]:
data['age'] = data['contract_year'] - data['TOP']

In [95]:
data=pd.get_dummies(data, columns=['district','property_type'])

In [91]:
data=data.drop(columns ='address')

In [ ]:
data['tenure'].apply(lambda x: